In [ ]:
# ============================================================
# HUMAN ACTIVITY RECOGNITION - CMPE 287
# Team: SANNS
# Members: Salma Ibrahim, Sana Al Hamimidi, Akmal Shaikh, Nicholas Faylor, Noah Scheuerman
# Date: 11/14/2025
# ============================================================

"""
This project classifies 18 different human activities using smartphone 
sensor data (accelerometer and gyroscope) from the KU-HAR dataset.

We use two machine learning models:
1. Random Forest Classifier
2. Support Vector Machine (SVM)

Dataset: 20,750 samples from 90 participants
Activities: Walking, Running, Sitting, Standing, Jumping, etc.
"""

# ============================================================
# SECTION 1: SETUP & IMPORTS
# ============================================================

# Data manipulation and analysis
import pandas as pd
import numpy as np

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Machine Learning - Model Selection
from sklearn.model_selection import train_test_split

# Machine Learning - Preprocessing
from sklearn.preprocessing import StandardScaler

# Machine Learning - Models
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC

# Machine Learning - Evaluation Metrics
from sklearn.metrics import (
    accuracy_score,
    precision_score,
    recall_score,
    f1_score,
    confusion_matrix,
    classification_report
)

# Utilities
import warnings
warnings.filterwarnings('ignore')

# Set random seed for reproducibility
RANDOM_STATE = 42
np.random.seed(RANDOM_STATE)

# Set visualization defaults
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

print("✓ All libraries imported successfully!")
print(f"✓ Random state set to: {RANDOM_STATE}")
print(f"✓ NumPy version: {np.__version__}")
print(f"✓ Pandas version: {pd.__version__}")
print("\nReady to load data!")

In [ ]:
# ============================================================
# SECTION 2: LOAD DATASET
# ============================================================

# Define activity labels (Class IDs 0-17)
activity_names = {
    0: 'Stand',
    1: 'Sit',
    2: 'Talk-sit',
    3: 'Talk-stand',
    4: 'Stand-sit',
    5: 'Lay',
    6: 'Lay-stand',
    7: 'Pick',
    8: 'Jump',
    9: 'Push-up',
    10: 'Sit-up',
    11: 'Walk',
    12: 'Walk-backward',
    13: 'Walk-circle',
    14: 'Run',
    15: 'Stair-up',
    16: 'Stair-down',
    17: 'Table-tennis'
}

print("Loading KU-HAR Time Domain Subsamples dataset...")
print("=" * 60)

# Load the dataset (no header in CSV)
# NOTE: Update the path to match where you saved the CSV file
df = pd.read_csv('data/time_domain_subsamples.csv', header=None)

print(f"✓ Dataset loaded successfully!")
print(f"\nDataset Shape: {df.shape}")
print(f"  - Total samples: {df.shape[0]:,}")
print(f"  - Total columns: {df.shape[1]:,}")

# Extract labels (column 1800 contains class IDs 0-17)
labels = df.iloc[:, 1800]

print(f"\n{'Activity Distribution:'}")
print("=" * 60)
class_distribution = labels.value_counts().sort_index()

for class_id, count in class_distribution.items():
    activity_name = activity_names[class_id]
    percentage = (count / len(labels)) * 100
    print(f"  {class_id:2d}. {activity_name:15s}: {count:4d} samples ({percentage:5.2f}%)")

print(f"\n{'Dataset Summary:'}")
print("=" * 60)
print(f"  - Sensor readings per sample: 1,800 (300 points × 6 axes)")
print(f"  - Sampling rate: 100 Hz")
print(f"  - Window duration: 3 seconds")
print(f"  - Number of activities: 18")
print(f"  - Total participants: 90")

# Verify data structure
print(f"\n{'Data Structure Verification:'}")
print("=" * 60)
print(f"  - Columns 0-299:      Accelerometer X")
print(f"  - Columns 300-599:    Accelerometer Y")
print(f"  - Columns 600-899:    Accelerometer Z")
print(f"  - Columns 900-1199:   Gyroscope X")
print(f"  - Columns 1200-1499:  Gyroscope Y")
print(f"  - Columns 1500-1799:  Gyroscope Z")
print(f"  - Column 1800:        Class ID (0-17)")
print(f"  - Column 1801:        Data length")
print(f"  - Column 1802:        Serial number")

print("\n✓ Ready for feature extraction!")

In [ ]:
# ============================================================
# SECTION 3: FEATURE EXTRACTION
# ============================================================

print("Starting feature extraction...")
print("=" * 60)
print("Converting 1,800 raw sensor values per sample into 24 statistical features")
print()

def extract_features(row):
    """
    Extract statistical features from time-series sensor data.
    
    For each of the 6 sensor axes (Accel X/Y/Z, Gyro X/Y/Z), we extract:
    - Mean: Average value over 3 seconds
    - Std: Standard deviation (variability)
    - Max: Maximum value
    - Min: Minimum value
    
    Total: 6 axes × 4 statistics = 24 features
    """
    features = {}
    
    # Accelerometer X (columns 0-299)
    accel_x = row[0:300].values
    features['accel_x_mean'] = np.mean(accel_x)
    features['accel_x_std'] = np.std(accel_x)
    features['accel_x_max'] = np.max(accel_x)
    features['accel_x_min'] = np.min(accel_x)
    
    # Accelerometer Y (columns 300-599)
    accel_y = row[300:600].values
    features['accel_y_mean'] = np.mean(accel_y)
    features['accel_y_std'] = np.std(accel_y)
    features['accel_y_max'] = np.max(accel_y)
    features['accel_y_min'] = np.min(accel_y)
    
    # Accelerometer Z (columns 600-899)
    accel_z = row[600:900].values
    features['accel_z_mean'] = np.mean(accel_z)
    features['accel_z_std'] = np.std(accel_z)
    features['accel_z_max'] = np.max(accel_z)
    features['accel_z_min'] = np.min(accel_z)
    
    # Gyroscope X (columns 900-1199)
    gyro_x = row[900:1200].values
    features['gyro_x_mean'] = np.mean(gyro_x)
    features['gyro_x_std'] = np.std(gyro_x)
    features['gyro_x_max'] = np.max(gyro_x)
    features['gyro_x_min'] = np.min(gyro_x)
    
    # Gyroscope Y (columns 1200-1499)
    gyro_y = row[1200:1500].values
    features['gyro_y_mean'] = np.mean(gyro_y)
    features['gyro_y_std'] = np.std(gyro_y)
    features['gyro_y_max'] = np.max(gyro_y)
    features['gyro_y_min'] = np.min(gyro_y)
    
    # Gyroscope Z (columns 1500-1799)
    gyro_z = row[1500:1800].values
    features['gyro_z_mean'] = np.mean(gyro_z)
    features['gyro_z_std'] = np.std(gyro_z)
    features['gyro_z_max'] = np.max(gyro_z)
    features['gyro_z_min'] = np.min(gyro_z)
    
    return features

# Extract features for all samples
print("Extracting features from all 20,750 samples...")
print("This may take a minute or two...\n")

feature_list = []
total_samples = len(df)

for idx, row in df.iterrows():
    feature_list.append(extract_features(row))
    
    # Progress update every 5,000 samples
    if (idx + 1) % 5000 == 0:
        progress = ((idx + 1) / total_samples) * 100
        print(f"  Progress: {idx + 1:,}/{total_samples:,} samples ({progress:.1f}%)")

print(f"  Progress: {total_samples:,}/{total_samples:,} samples (100.0%)")

# Create feature DataFrame
features_df = pd.DataFrame(feature_list)

print(f"\n✓ Feature extraction complete!")
print("=" * 60)
print(f"Feature Matrix Shape: {features_df.shape}")
print(f"  - Samples: {features_df.shape[0]:,}")
print(f"  - Features per sample: {features_df.shape[1]}")

print(f"\nExtracted Features:")
print("-" * 60)
for i, col in enumerate(features_df.columns, 1):
    print(f"  {i:2d}. {col}")

print(f"\nFeature Statistics (first 5 features):")
print("-" * 60)
print(features_df.iloc[:, :5].describe())

print("\n✓ Ready for train-test split!")

In [ ]:
# ============================================================
# SECTION 4: TRAIN-TEST SPLIT
# ============================================================

print("Splitting data into training and testing sets...")
print("=" * 60)

# Split: 80% training, 20% testing
# Stratify ensures each activity is proportionally represented in both sets
X_train, X_test, y_train, y_test = train_test_split(
    features_df,           # Feature matrix (24 features)
    labels,                # Class labels (0-17)
    test_size=0.2,         # 20% for testing
    stratify=labels,       # Maintain class distribution
    random_state=RANDOM_STATE  # Reproducibility
)

print(f"✓ Data split complete!")
print()

# Display split information
print(f"{'Dataset Split Summary:'}")
print("-" * 60)
print(f"  Training samples:   {len(X_train):,} ({len(X_train)/len(features_df)*100:.1f}%)")
print(f"  Testing samples:    {len(X_test):,} ({len(X_test)/len(features_df)*100:.1f}%)")
print(f"  Total samples:      {len(features_df):,}")
print()
print(f"  Features per sample: {X_train.shape[1]}")

# Verify stratification worked - show class distribution in train/test
print(f"\n{'Class Distribution Verification:'}")
print("=" * 60)
print(f"{'Activity':<20} {'Train Count':<15} {'Test Count':<15} {'Train %':<10} {'Test %'}")
print("-" * 60)

train_dist = y_train.value_counts().sort_index()
test_dist = y_test.value_counts().sort_index()

for class_id in range(18):
    activity = activity_names[class_id]
    train_count = train_dist.get(class_id, 0)
    test_count = test_dist.get(class_id, 0)
    train_pct = (train_count / len(y_train)) * 100
    test_pct = (test_count / len(y_test)) * 100
    
    print(f"{activity:<20} {train_count:<15} {test_count:<15} {train_pct:<10.2f} {test_pct:.2f}")

print()
print("✓ Stratification successful - distributions are proportional!")
print("\n✓ Ready for feature scaling!")


In [ ]:
# ============================================================
# SECTION 5: FEATURE SCALING (FIXED WITH ROBUST SCALER)
# ============================================================

from sklearn.preprocessing import RobustScaler

print("Scaling features using RobustScaler...")
print("=" * 60)
print("Why RobustScaler? It handles outliers better than StandardScaler.")
print("Uses median and IQR instead of mean and std.\n")

# Initialize RobustScaler (robust to outliers)
scaler = RobustScaler()

# Fit on training data only, then transform both sets
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print("✓ Feature scaling complete!")
print()

# Show scaling effect
print(f"{'Scaling Effect (example: accel_x_mean):'}")
print("-" * 60)
print(f"Before scaling:")
print(f"  Training median: {X_train.iloc[:, 0].median():.4f}")
print(f"  Training IQR:    {X_train.iloc[:, 0].quantile(0.75) - X_train.iloc[:, 0].quantile(0.25):.4f}")
print()
print(f"After scaling:")
print(f"  Training median: {X_train_scaled[:, 0].mean():.4f}")
print(f"  Data range: [{X_train_scaled.min():.4f}, {X_train_scaled.max():.4f}]")
print()
print("Features now scaled robustly (handles outliers)")

print("\n✓ Ready to train models!")

In [ ]:
# ============================================================
# SECTION 6: TRAIN RANDOM FOREST MODEL
# ============================================================

print("Training Random Forest Classifier...")
print("=" * 60)

# Initialize Random Forest
rf_model = RandomForestClassifier(
    n_estimators=100,        # 100 decision trees
    max_depth=None,          # No limit on tree depth
    min_samples_split=2,     # Minimum samples to split a node
    random_state=RANDOM_STATE,
    n_jobs=-1                # Use all CPU cores
)

print(f"Model configuration:")
print(f"  - Number of trees: 100")
print(f"  - Max depth: Unlimited")
print(f"  - Random state: {RANDOM_STATE}")
print()

# Train the model
print("Training in progress...")
import time
start_time = time.time()

rf_model.fit(X_train_scaled, y_train)

training_time = time.time() - start_time
print(f"✓ Training complete! Time: {training_time:.2f} seconds")
print()

# Make predictions
print("Making predictions on test set...")
y_pred_rf = rf_model.predict(X_test_scaled)
y_pred_proba_rf = rf_model.predict_proba(X_test_scaled)

print("✓ Predictions complete!")
print()

# Quick accuracy preview
rf_accuracy = accuracy_score(y_test, y_pred_rf)
print("=" * 60)
print(f"RANDOM FOREST - QUICK PREVIEW")
print("=" * 60)
print(f"Test Set Accuracy: {rf_accuracy*100:.2f}%")
print()
print("(Detailed evaluation in next sections)")

print("\n✓ Random Forest model trained successfully!")
print("✓ Ready to train SVM model!")

In [ ]:
# ============================================================
# SECTION 7: TRAIN SVM MODEL (WITH ROBUST SCALING)
# ============================================================

print("Training SVM Classifier (with RobustScaler data)...")
print("=" * 60)

# Use linear kernel - simpler and more stable
svm_model = SVC(
    kernel='linear',
    C=1.0,
    random_state=RANDOM_STATE,
    max_iter=5000
)

print(f"Model configuration:")
print(f"  - Kernel: Linear")
print(f"  - C: 1.0")
print(f"  - Data: RobustScaler (handles outliers)")
print()

# Train
print("Training in progress...")
start_time = time.time()

svm_model.fit(X_train_scaled, y_train)

training_time = time.time() - start_time
print(f"✓ Training complete! Time: {training_time:.2f} seconds")
print()

# Predict
print("Making predictions on test set...")
y_pred_svm = svm_model.predict(X_test_scaled)

print("✓ Predictions complete!")
print()

# Accuracy
svm_accuracy = accuracy_score(y_test, y_pred_svm)
print("=" * 60)
print(f"SVM - RESULTS")
print("=" * 60)
print(f"Test Set Accuracy: {svm_accuracy*100:.2f}%")
print(f"Classes predicted: {len(np.unique(y_pred_svm))}/18")
print()

# Compare
print("=" * 60)
print(f"MODEL COMPARISON")
print("=" * 60)
print(f"Random Forest Accuracy: {rf_accuracy*100:.2f}%")
print(f"SVM Accuracy:           {svm_accuracy*100:.2f}%")
print(f"Difference:             {abs(rf_accuracy - svm_accuracy)*100:.2f}%")

if rf_accuracy > svm_accuracy:
    print(f"\n→ Random Forest performs better by {(rf_accuracy - svm_accuracy)*100:.2f}%")
elif svm_accuracy > rf_accuracy:
    print(f"\n→ SVM performs better by {(svm_accuracy - rf_accuracy)*100:.2f}%")

print("\n✓ Both models trained successfully!")

In [ ]:
# ============================================================
# SECTION 8: DETAILED EVALUATION METRICS
# ============================================================

print("Generating detailed evaluation metrics for both models...")
print("=" * 60)
print()

# ============================================================
# RANDOM FOREST - DETAILED METRICS
# ============================================================

print("=" * 80)
print("RANDOM FOREST - DETAILED CLASSIFICATION REPORT")
print("=" * 80)
print()

rf_report = classification_report(
    y_test, 
    y_pred_rf,
    target_names=[activity_names[i] for i in range(18)],
    digits=4,
    output_dict=False
)
print(rf_report)

# Get per-class metrics as dictionary for later analysis
rf_report_dict = classification_report(
    y_test, 
    y_pred_rf,
    target_names=[activity_names[i] for i in range(18)],
    digits=4,
    output_dict=True
)

print("\n" + "=" * 80)
print("RANDOM FOREST - PERFORMANCE SUMMARY")
print("=" * 80)

# Overall metrics
print(f"\nOverall Metrics:")
print(f"  Accuracy:          {rf_accuracy*100:.2f}%")
print(f"  Macro Avg F1:      {rf_report_dict['macro avg']['f1-score']:.4f}")
print(f"  Weighted Avg F1:   {rf_report_dict['weighted avg']['f1-score']:.4f}")

# Best performing activities
print(f"\nTop 5 Best Performing Activities (by F1-score):")
activity_scores_rf = [(activity_names[i], rf_report_dict[activity_names[i]]['f1-score']) 
                      for i in range(18)]
activity_scores_rf.sort(key=lambda x: x[1], reverse=True)

for i, (activity, score) in enumerate(activity_scores_rf[:5], 1):
    print(f"  {i}. {activity:<20} F1: {score:.4f}")

# Worst performing activities
print(f"\nBottom 5 Challenging Activities (by F1-score):")
for i, (activity, score) in enumerate(activity_scores_rf[-5:], 1):
    print(f"  {i}. {activity:<20} F1: {score:.4f}")

print("\n" + "-" * 80)

# ============================================================
# SVM - DETAILED METRICS
# ============================================================

print("\n" + "=" * 80)
print("SVM - DETAILED CLASSIFICATION REPORT")
print("=" * 80)
print()

svm_report = classification_report(
    y_test, 
    y_pred_svm,
    target_names=[activity_names[i] for i in range(18)],
    digits=4,
    output_dict=False,
    zero_division=0  # Handle missing classes gracefully
)
print(svm_report)

# Get per-class metrics as dictionary
svm_report_dict = classification_report(
    y_test, 
    y_pred_svm,
    target_names=[activity_names[i] for i in range(18)],
    digits=4,
    output_dict=True,
    zero_division=0
)

print("\n" + "=" * 80)
print("SVM - PERFORMANCE SUMMARY")
print("=" * 80)

# Overall metrics
print(f"\nOverall Metrics:")
print(f"  Accuracy:          {svm_accuracy*100:.2f}%")
print(f"  Macro Avg F1:      {svm_report_dict['macro avg']['f1-score']:.4f}")
print(f"  Weighted Avg F1:   {svm_report_dict['weighted avg']['f1-score']:.4f}")

# Best performing activities
print(f"\nTop 5 Best Performing Activities (by F1-score):")
activity_scores_svm = [(activity_names[i], svm_report_dict[activity_names[i]]['f1-score']) 
                       for i in range(18)]
activity_scores_svm.sort(key=lambda x: x[1], reverse=True)

for i, (activity, score) in enumerate(activity_scores_svm[:5], 1):
    print(f"  {i}. {activity:<20} F1: {score:.4f}")

# Worst performing activities
print(f"\nBottom 5 Challenging Activities (by F1-score):")
for i, (activity, score) in enumerate(activity_scores_svm[-5:], 1):
    print(f"  {i}. {activity:<20} F1: {score:.4f}")

print("\n" + "-" * 80)

# ============================================================
# COMPARATIVE ANALYSIS
# ============================================================

print("\n" + "=" * 80)
print("COMPARATIVE ANALYSIS - RF vs SVM")
print("=" * 80)

print(f"\n{'Activity':<20} {'RF F1':<12} {'SVM F1':<12} {'Difference':<12} {'Winner'}")
print("-" * 80)

for i in range(18):
    activity = activity_names[i]
    rf_f1 = rf_report_dict[activity]['f1-score']
    svm_f1 = svm_report_dict[activity]['f1-score']
    diff = rf_f1 - svm_f1
    winner = "RF" if rf_f1 > svm_f1 else ("SVM" if svm_f1 > rf_f1 else "Tie")
    
    print(f"{activity:<20} {rf_f1:<12.4f} {svm_f1:<12.4f} {diff:>+11.4f} {winner}")

# Summary statistics
print("\n" + "=" * 80)
print("KEY INSIGHTS")
print("=" * 80)

# Count how many activities each model wins
rf_wins = sum(1 for i in range(18) if rf_report_dict[activity_names[i]]['f1-score'] > 
              svm_report_dict[activity_names[i]]['f1-score'])
svm_wins = sum(1 for i in range(18) if svm_report_dict[activity_names[i]]['f1-score'] > 
               rf_report_dict[activity_names[i]]['f1-score'])
ties = 18 - rf_wins - svm_wins

print(f"\nPer-Activity Performance:")
print(f"  Random Forest wins: {rf_wins}/18 activities")
print(f"  SVM wins:           {svm_wins}/18 activities")
print(f"  Ties:               {ties}/18 activities")

# Average performance gap
avg_gap = np.mean([rf_report_dict[activity_names[i]]['f1-score'] - 
                   svm_report_dict[activity_names[i]]['f1-score'] 
                   for i in range(18)])
print(f"\nAverage F1-score gap: {avg_gap:+.4f} (positive = RF better)")

# Find activities where SVM did surprisingly well
print(f"\nActivities where SVM performed competitively (within 5% of RF):")
competitive = []
for i in range(18):
    activity = activity_names[i]
    rf_f1 = rf_report_dict[activity]['f1-score']
    svm_f1 = svm_report_dict[activity]['f1-score']
    if abs(rf_f1 - svm_f1) <= 0.05:
        competitive.append((activity, rf_f1, svm_f1))

if competitive:
    for activity, rf_f1, svm_f1 in competitive:
        print(f"  - {activity:<20} RF: {rf_f1:.4f}, SVM: {svm_f1:.4f}")
else:
    print("  None - Random Forest dominates across all activities")

print("\n✓ Detailed evaluation complete!")
print("✓ Ready for confusion matrix visualization!")

In [ ]:
# ============================================================
# SECTION 9: CONFUSION MATRIX VISUALIZATION
# ============================================================

print("Generating confusion matrices for both models...")
print("=" * 60)
print()

# Calculate confusion matrices
cm_rf = confusion_matrix(y_test, y_pred_rf)
cm_svm = confusion_matrix(y_test, y_pred_svm)

# Activity names for labels
activity_labels = [activity_names[i] for i in range(18)]

# ============================================================
# PLOT SIDE-BY-SIDE CONFUSION MATRICES
# ============================================================

fig, axes = plt.subplots(1, 2, figsize=(24, 10))

# Random Forest Confusion Matrix
sns.heatmap(
    cm_rf,
    annot=True,
    fmt='d',
    cmap='Blues',
    xticklabels=activity_labels,
    yticklabels=activity_labels,
    ax=axes[0],
    cbar_kws={'label': 'Number of Samples'}
)
axes[0].set_xlabel('Predicted Activity', fontsize=12, fontweight='bold')
axes[0].set_ylabel('Actual Activity', fontsize=12, fontweight='bold')
axes[0].set_title(f'Random Forest Confusion Matrix\nAccuracy: {rf_accuracy*100:.2f}%', 
                  fontsize=14, fontweight='bold')
axes[0].tick_params(axis='x', rotation=45)
axes[0].tick_params(axis='y', rotation=0)

# SVM Confusion Matrix
sns.heatmap(
    cm_svm,
    annot=True,
    fmt='d',
    cmap='Greens',
    xticklabels=activity_labels,
    yticklabels=activity_labels,
    ax=axes[1],
    cbar_kws={'label': 'Number of Samples'}
)
axes[1].set_xlabel('Predicted Activity', fontsize=12, fontweight='bold')
axes[1].set_ylabel('Actual Activity', fontsize=12, fontweight='bold')
axes[1].set_title(f'SVM Confusion Matrix\nAccuracy: {svm_accuracy*100:.2f}%', 
                  fontsize=14, fontweight='bold')
axes[1].tick_params(axis='x', rotation=45)
axes[1].tick_params(axis='y', rotation=0)

plt.tight_layout()
plt.savefig('confusion_matrices_comparison.png', dpi=300, bbox_inches='tight')
plt.show()

print("✓ Confusion matrices plotted!")
print("✓ Saved as: confusion_matrices_comparison.png")
print()

# ============================================================
# ANALYZE CONFUSION PATTERNS
# ============================================================

print("=" * 80)
print("CONFUSION PATTERN ANALYSIS")
print("=" * 80)
print()

# Random Forest - Most Common Misclassifications
print("RANDOM FOREST - Top 10 Misclassification Pairs:")
print("-" * 80)

rf_errors = []
for i in range(18):
    for j in range(18):
        if i != j and cm_rf[i][j] > 0:
            rf_errors.append((activity_names[i], activity_names[j], cm_rf[i][j]))

rf_errors.sort(key=lambda x: x[2], reverse=True)

for idx, (true_class, pred_class, count) in enumerate(rf_errors[:10], 1):
    percentage = (count / cm_rf[list(activity_names.values()).index(true_class), :].sum()) * 100
    print(f"  {idx:2d}. {true_class:<15} → {pred_class:<15} : {count:3d} errors ({percentage:5.2f}%)")

print()

# SVM - Most Common Misclassifications
print("SVM - Top 10 Misclassification Pairs:")
print("-" * 80)

svm_errors = []
for i in range(18):
    for j in range(18):
        if i != j and cm_svm[i][j] > 0:
            svm_errors.append((activity_names[i], activity_names[j], cm_svm[i][j]))

svm_errors.sort(key=lambda x: x[2], reverse=True)

for idx, (true_class, pred_class, count) in enumerate(svm_errors[:10], 1):
    percentage = (count / cm_svm[list(activity_names.values()).index(true_class), :].sum()) * 100
    print(f"  {idx:2d}. {true_class:<15} → {pred_class:<15} : {count:3d} errors ({percentage:5.2f}%)")

print()

# ============================================================
# PER-CLASS ACCURACY FROM CONFUSION MATRIX
# ============================================================

print("=" * 80)
print("PER-CLASS ACCURACY (from Confusion Matrix)")
print("=" * 80)
print()

print(f"{'Activity':<20} {'RF Accuracy':<15} {'SVM Accuracy':<15} {'Difference'}")
print("-" * 80)

for i in range(18):
    activity = activity_names[i]
    
    # Calculate per-class accuracy (diagonal / row sum)
    rf_class_acc = cm_rf[i][i] / cm_rf[i, :].sum() if cm_rf[i, :].sum() > 0 else 0
    svm_class_acc = cm_svm[i][i] / cm_svm[i, :].sum() if cm_svm[i, :].sum() > 0 else 0
    diff = rf_class_acc - svm_class_acc
    
    print(f"{activity:<20} {rf_class_acc*100:>6.2f}%        {svm_class_acc*100:>6.2f}%        {diff*100:>+6.2f}%")

print()

# ============================================================
# KEY INSIGHTS FROM CONFUSION MATRICES
# ============================================================

print("=" * 80)
print("KEY INSIGHTS")
print("=" * 80)
print()

# Find activities that are commonly confused with each other (both models)
print("Activities Commonly Confused by BOTH Models:")
print("-" * 80)

common_confusions = []
for i in range(18):
    for j in range(18):
        if i != j:
            # If both models confuse these activities significantly
            rf_conf = cm_rf[i][j]
            svm_conf = cm_svm[i][j]
            if rf_conf >= 5 and svm_conf >= 5:  # At least 5 errors in both
                common_confusions.append((activity_names[i], activity_names[j], rf_conf, svm_conf))

if common_confusions:
    for true_act, pred_act, rf_err, svm_err in common_confusions:
        print(f"  {true_act:<15} → {pred_act:<15} : RF={rf_err:3d} errors, SVM={svm_err:3d} errors")
else:
    print("  No common confusion patterns found")

print()

# Activities with perfect classification (RF)
perfect_rf = [activity_names[i] for i in range(18) if cm_rf[i][i] == cm_rf[i, :].sum()]
if perfect_rf:
    print(f"Activities with PERFECT classification by Random Forest:")
    for activity in perfect_rf:
        print(f"  ✓ {activity}")
    print()

# Activities with perfect classification (SVM)
perfect_svm = [activity_names[i] for i in range(18) if cm_svm[i][i] == cm_svm[i, :].sum()]
if perfect_svm:
    print(f"Activities with PERFECT classification by SVM:")
    for activity in perfect_svm:
        print(f"  ✓ {activity}")
    print()
else:
    print("No activities perfectly classified by SVM")
    print()

# Most problematic activity for each model
print("Most Challenging Activity for Each Model:")
print("-" * 80)

rf_worst_idx = np.argmin([cm_rf[i][i] / cm_rf[i, :].sum() for i in range(18)])
svm_worst_idx = np.argmin([cm_svm[i][i] / cm_svm[i, :].sum() if cm_svm[i, :].sum() > 0 else 1 
                           for i in range(18)])

rf_worst_acc = cm_rf[rf_worst_idx][rf_worst_idx] / cm_rf[rf_worst_idx, :].sum()
svm_worst_acc = cm_svm[svm_worst_idx][svm_worst_idx] / cm_svm[svm_worst_idx, :].sum() if cm_svm[svm_worst_idx, :].sum() > 0 else 0

print(f"  Random Forest: {activity_names[rf_worst_idx]} ({rf_worst_acc*100:.2f}% accuracy)")
print(f"  SVM:           {activity_names[svm_worst_idx]} ({svm_worst_acc*100:.2f}% accuracy)")

print()
print("✓ Confusion matrix analysis complete!")
print("✓ Ready for feature importance analysis!")

In [ ]:
# ============================================================
# SECTION 10: FEATURE IMPORTANCE ANALYSIS (RANDOM FOREST)
# ============================================================

print("Analyzing feature importance from Random Forest...")
print("=" * 60)
print()

# Get feature importance from Random Forest
feature_importance = pd.DataFrame({
    'feature': features_df.columns,
    'importance': rf_model.feature_importances_
}).sort_values('importance', ascending=False)

print("=" * 80)
print("FEATURE IMPORTANCE RANKING")
print("=" * 80)
print()

print(f"{'Rank':<6} {'Feature':<25} {'Importance':<15} {'Cumulative %'}")
print("-" * 80)

cumulative = 0
for idx, row in feature_importance.iterrows():
    cumulative += row['importance']
    print(f"{feature_importance.index.get_loc(idx)+1:<6} {row['feature']:<25} {row['importance']:<15.6f} {cumulative*100:>6.2f}%")

print()

# ============================================================
# VISUALIZE TOP FEATURES
# ============================================================

print("Plotting feature importance...")
print()

fig, axes = plt.subplots(2, 1, figsize=(12, 10))

# Plot 1: Top 15 Features (Bar Chart)
top_15 = feature_importance.head(15)

axes[0].barh(range(len(top_15)), top_15['importance'], color='steelblue')
axes[0].set_yticks(range(len(top_15)))
axes[0].set_yticklabels(top_15['feature'])
axes[0].invert_yaxis()
axes[0].set_xlabel('Importance Score', fontsize=11, fontweight='bold')
axes[0].set_title('Top 15 Most Important Features', fontsize=13, fontweight='bold')
axes[0].grid(axis='x', alpha=0.3)

# Add importance values on bars
for i, (idx, row) in enumerate(top_15.iterrows()):
    axes[0].text(row['importance'], i, f" {row['importance']:.4f}", 
                va='center', fontsize=9)

# Plot 2: Feature Importance by Category
axes[1].bar(range(len(feature_importance)), feature_importance['importance'], 
           color='coral', alpha=0.7)
axes[1].set_xlabel('Feature Rank', fontsize=11, fontweight='bold')
axes[1].set_ylabel('Importance Score', fontsize=11, fontweight='bold')
axes[1].set_title('Feature Importance Distribution (All 24 Features)', 
                 fontsize=13, fontweight='bold')
axes[1].set_xticks(range(0, 24, 2))
axes[1].set_xticklabels(range(1, 25, 2))
axes[1].grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.savefig('feature_importance.png', dpi=300, bbox_inches='tight')
plt.show()

print("✓ Feature importance visualization saved!")
print()

# ============================================================
# ANALYZE BY SENSOR AND STATISTIC TYPE
# ============================================================

print("=" * 80)
print("FEATURE IMPORTANCE BY SENSOR TYPE")
print("=" * 80)
print()

# Group by sensor (accel vs gyro)
accel_importance = feature_importance[feature_importance['feature'].str.contains('accel')]['importance'].sum()
gyro_importance = feature_importance[feature_importance['feature'].str.contains('gyro')]['importance'].sum()

print(f"Accelerometer features total importance: {accel_importance:.6f} ({accel_importance*100:.2f}%)")
print(f"Gyroscope features total importance:     {gyro_importance:.6f} ({gyro_importance*100:.2f}%)")
print()

if accel_importance > gyro_importance:
    print(f"→ Accelerometer is {accel_importance/gyro_importance:.2f}x more important than Gyroscope")
else:
    print(f"→ Gyroscope is {gyro_importance/accel_importance:.2f}x more important than Accelerometer")

print()

# ============================================================
# ANALYZE BY STATISTIC TYPE
# ============================================================

print("=" * 80)
print("FEATURE IMPORTANCE BY STATISTIC TYPE")
print("=" * 80)
print()

mean_importance = feature_importance[feature_importance['feature'].str.contains('mean')]['importance'].sum()
std_importance = feature_importance[feature_importance['feature'].str.contains('std')]['importance'].sum()
max_importance = feature_importance[feature_importance['feature'].str.contains('max')]['importance'].sum()
min_importance = feature_importance[feature_importance['feature'].str.contains('min')]['importance'].sum()

print(f"Mean features total importance: {mean_importance:.6f} ({mean_importance*100:.2f}%)")
print(f"Std features total importance:  {std_importance:.6f} ({std_importance*100:.2f}%)")
print(f"Max features total importance:  {max_importance:.6f} ({max_importance*100:.2f}%)")
print(f"Min features total importance:  {min_importance:.6f} ({min_importance*100:.2f}%)")
print()

# Find most important statistic
stats = {
    'Mean': mean_importance,
    'Std': std_importance,
    'Max': max_importance,
    'Min': min_importance
}
most_important_stat = max(stats, key=stats.get)
print(f"→ Most important statistic type: {most_important_stat} ({stats[most_important_stat]*100:.2f}%)")

print()

# ============================================================
# ANALYZE BY AXIS
# ============================================================

print("=" * 80)
print("FEATURE IMPORTANCE BY SENSOR AXIS")
print("=" * 80)
print()

x_importance = feature_importance[feature_importance['feature'].str.contains('_x_')]['importance'].sum()
y_importance = feature_importance[feature_importance['feature'].str.contains('_y_')]['importance'].sum()
z_importance = feature_importance[feature_importance['feature'].str.contains('_z_')]['importance'].sum()

print(f"X-axis features total importance: {x_importance:.6f} ({x_importance*100:.2f}%)")
print(f"Y-axis features total importance: {y_importance:.6f} ({y_importance*100:.2f}%)")
print(f"Z-axis features total importance: {z_importance:.6f} ({z_importance*100:.2f}%)")
print()

axes_dict = {'X': x_importance, 'Y': y_importance, 'Z': z_importance}
most_important_axis = max(axes_dict, key=axes_dict.get)
print(f"→ Most important axis: {most_important_axis}-axis ({axes_dict[most_important_axis]*100:.2f}%)")

print()

# ============================================================
# KEY INSIGHTS
# ============================================================

print("=" * 80)
print("KEY INSIGHTS")
print("=" * 80)
print()

top_feature = feature_importance.iloc[0]
print(f"Most Important Feature: {top_feature['feature']}")
print(f"  Importance: {top_feature['importance']:.6f} ({top_feature['importance']*100:.2f}%)")
print()

# Features needed for 80% importance
cumulative_importance = 0
features_for_80 = 0
for idx, row in feature_importance.iterrows():
    cumulative_importance += row['importance']
    features_for_80 += 1
    if cumulative_importance >= 0.80:
        break

print(f"Number of features needed to reach 80% cumulative importance: {features_for_80}/24")
print(f"  (This means {24 - features_for_80} features contribute only 20% to predictions)")
print()

# Find least important features
bottom_5 = feature_importance.tail(5)
print("5 Least Important Features:")
for idx, row in bottom_5.iterrows():
    print(f"  - {row['feature']:<25} : {row['importance']:.6f}")

print()
print("✓ Feature importance analysis complete!")
print("✓ Ready for final visualizations and summary!")

In [ ]:
# ============================================================
# SECTION 11: FINAL MODEL COMPARISON VISUALIZATIONS
# ============================================================

print("Creating final comparison visualizations...")
print("=" * 60)
print()

# ============================================================
# VISUALIZATION 1: Overall Accuracy Comparison
# ============================================================

fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# Plot 1: Overall Accuracy Bar Chart
models = ['Random Forest', 'SVM']
accuracies = [rf_accuracy * 100, svm_accuracy * 100]
colors = ['steelblue', 'coral']

bars = axes[0, 0].bar(models, accuracies, color=colors, alpha=0.8, edgecolor='black', linewidth=2)
axes[0, 0].set_ylabel('Accuracy (%)', fontsize=12, fontweight='bold')
axes[0, 0].set_title('Overall Model Accuracy Comparison', fontsize=14, fontweight='bold')
axes[0, 0].set_ylim(0, 100)
axes[0, 0].grid(axis='y', alpha=0.3)

# Add value labels on bars
for bar, acc in zip(bars, accuracies):
    height = bar.get_height()
    axes[0, 0].text(bar.get_x() + bar.get_width()/2., height + 1,
                    f'{acc:.2f}%',
                    ha='center', va='bottom', fontsize=14, fontweight='bold')

# Add baseline reference line
axes[0, 0].axhline(y=100/18, color='red', linestyle='--', linewidth=2, 
                   label=f'Random Guess ({100/18:.1f}%)')
axes[0, 0].legend()

# ============================================================
# Plot 2: Per-Class Accuracy Comparison
# ============================================================

# Calculate per-class accuracies from confusion matrices
rf_class_acc = [cm_rf[i][i] / cm_rf[i, :].sum() * 100 for i in range(18)]
svm_class_acc = [cm_svm[i][i] / cm_svm[i, :].sum() * 100 if cm_svm[i, :].sum() > 0 else 0 
                 for i in range(18)]

x = np.arange(18)
width = 0.35

bars1 = axes[0, 1].bar(x - width/2, rf_class_acc, width, label='Random Forest', 
                       color='steelblue', alpha=0.8)
bars2 = axes[0, 1].bar(x + width/2, svm_class_acc, width, label='SVM', 
                       color='coral', alpha=0.8)

axes[0, 1].set_xlabel('Activity', fontsize=11, fontweight='bold')
axes[0, 1].set_ylabel('Accuracy (%)', fontsize=11, fontweight='bold')
axes[0, 1].set_title('Per-Activity Accuracy: RF vs SVM', fontsize=14, fontweight='bold')
axes[0, 1].set_xticks(x)
axes[0, 1].set_xticklabels(range(18), fontsize=9)
axes[0, 1].legend()
axes[0, 1].grid(axis='y', alpha=0.3)
axes[0, 1].set_ylim(0, 105)

# ============================================================
# Plot 3: Performance Metrics Comparison (Precision, Recall, F1)
# ============================================================

metrics = ['Precision', 'Recall', 'F1-Score']
rf_metrics = [
    rf_report_dict['weighted avg']['precision'] * 100,
    rf_report_dict['weighted avg']['recall'] * 100,
    rf_report_dict['weighted avg']['f1-score'] * 100
]
svm_metrics = [
    svm_report_dict['weighted avg']['precision'] * 100,
    svm_report_dict['weighted avg']['recall'] * 100,
    svm_report_dict['weighted avg']['f1-score'] * 100
]

x_metrics = np.arange(len(metrics))
width = 0.35

bars1 = axes[1, 0].bar(x_metrics - width/2, rf_metrics, width, 
                       label='Random Forest', color='steelblue', alpha=0.8)
bars2 = axes[1, 0].bar(x_metrics + width/2, svm_metrics, width, 
                       label='SVM', color='coral', alpha=0.8)

axes[1, 0].set_ylabel('Score (%)', fontsize=11, fontweight='bold')
axes[1, 0].set_title('Performance Metrics Comparison (Weighted Avg)', 
                     fontsize=14, fontweight='bold')
axes[1, 0].set_xticks(x_metrics)
axes[1, 0].set_xticklabels(metrics)
axes[1, 0].legend()
axes[1, 0].set_ylim(0, 100)
axes[1, 0].grid(axis='y', alpha=0.3)

# Add value labels
for bars in [bars1, bars2]:
    for bar in bars:
        height = bar.get_height()
        axes[1, 0].text(bar.get_x() + bar.get_width()/2., height + 1,
                       f'{height:.1f}%',
                       ha='center', va='bottom', fontsize=9)

# ============================================================
# Plot 4: Training Time and Model Characteristics
# ============================================================

# Summary statistics table
summary_data = {
    'Metric': [
        'Overall Accuracy',
        'Weighted F1-Score',
        'Macro F1-Score',
        'Activities Won',
        'Perfect Classifications'
    ],
    'Random Forest': [
        f'{rf_accuracy*100:.2f}%',
        f'{rf_report_dict["weighted avg"]["f1-score"]:.4f}',
        f'{rf_report_dict["macro avg"]["f1-score"]:.4f}',
        '17/18',
        '0'  # Update if any were perfect
    ],
    'SVM': [
        f'{svm_accuracy*100:.2f}%',
        f'{svm_report_dict["weighted avg"]["f1-score"]:.4f}',
        f'{svm_report_dict["macro avg"]["f1-score"]:.4f}',
        '1/18',
        '0'
    ]
}

# Create table
axes[1, 1].axis('tight')
axes[1, 1].axis('off')

table = axes[1, 1].table(cellText=[[summary_data['Metric'][i], 
                                    summary_data['Random Forest'][i],
                                    summary_data['SVM'][i]] 
                                   for i in range(len(summary_data['Metric']))],
                        colLabels=['Metric', 'Random Forest', 'SVM'],
                        cellLoc='center',
                        loc='center',
                        colWidths=[0.4, 0.3, 0.3])

table.auto_set_font_size(False)
table.set_fontsize(10)
table.scale(1, 2.5)

# Style header row
for i in range(3):
    table[(0, i)].set_facecolor('#4472C4')
    table[(0, i)].set_text_props(weight='bold', color='white')

# Alternate row colors
for i in range(1, len(summary_data['Metric']) + 1):
    if i % 2 == 0:
        for j in range(3):
            table[(i, j)].set_facecolor('#E7E6E6')

axes[1, 1].set_title('Model Performance Summary', fontsize=14, fontweight='bold', pad=20)

plt.tight_layout()
plt.savefig('final_model_comparison.png', dpi=300, bbox_inches='tight')
plt.show()

print("✓ Final comparison visualizations saved!")
print()

# ============================================================
# FINAL SUMMARY STATISTICS
# ============================================================

print("=" * 80)
print("FINAL PROJECT SUMMARY")
print("=" * 80)
print()

print("Dataset Information:")
print(f"  Total Samples:               20,750")
print(f"  Number of Activities:        18")
print(f"  Features per Sample:         24 (extracted from 1,800 sensor readings)")
print(f"  Training Samples:            16,600 (80%)")
print(f"  Testing Samples:             4,150 (20%)")
print()

print("Model Performance:")
print(f"  Random Forest Accuracy:      {rf_accuracy*100:.2f}%")
print(f"  SVM Accuracy:                {svm_accuracy*100:.2f}%")
print(f"  Performance Gap:             {(rf_accuracy - svm_accuracy)*100:.2f}%")
print()

print("Random Forest Details:")
print(f"  Best Activity:               {activity_scores_rf[0][0]} (F1: {activity_scores_rf[0][1]:.4f})")
print(f"  Worst Activity:              {activity_scores_rf[-1][0]} (F1: {activity_scores_rf[-1][1]:.4f})")
print(f"  Activities Won (vs SVM):     17/18")
print(f"  Weighted F1-Score:           {rf_report_dict['weighted avg']['f1-score']:.4f}")
print()

print("SVM Details:")
print(f"  Best Activity:               {activity_scores_svm[0][0]} (F1: {activity_scores_svm[0][1]:.4f})")
print(f"  Worst Activity:              {activity_scores_svm[-1][0]} (F1: {activity_scores_svm[-1][1]:.4f})")
print(f"  Activities Won (vs RF):      1/18 (Walk-backward)")
print(f"  Weighted F1-Score:           {svm_report_dict['weighted avg']['f1-score']:.4f}")
print()

print("Key Findings:")
print("  1. Random Forest significantly outperforms SVM (93.78% vs 71.78%)")
print("  2. Ensemble methods handle complex activity patterns better than linear classifiers")
print("  3. Standard deviation features are most important (44.95% total importance)")
print("  4. Gyroscope and Accelerometer contribute equally (~50% each)")
print("  5. SVM failed completely on 'Run' activity (0% F1-score)")
print("  6. Both models struggle with similar static activities (Stand, Sit, Lay)")
print()

print("=" * 80)
print("✓ Project analysis complete!")
print("✓ Ready to save models and conclude!")
print("=" * 80)

In [ ]:
# ============================================================
# SECTION 12: SAVE MODELS
# ============================================================

import joblib

print("Saving trained models...")

# Save Random Forest
joblib.dump(rf_model, 'random_forest_model.pkl')
print("✓ Random Forest saved as: random_forest_model.pkl")

# Save SVM
joblib.dump(svm_model, 'svm_model.pkl')
print("✓ SVM saved as: svm_model.pkl")

# Save Scaler (important!)
joblib.dump(scaler, 'scaler.pkl')
print("✓ Scaler saved as: scaler.pkl")

print("\n✓ All models saved successfully!")